In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

c:\Users\vaish\Desktop\chatbot_LLM_app\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
token= os.getenv("RUNPOD_TOKEN") 
open_ai_base_url = os.getenv("RUNPOD_EMBEDDING_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

# Try out embeddings

In [4]:
output = client.embeddings.create(input = ["hello there"],model=model_name)
embedings = output.data[0].embedding
print(embedings)

[-0.0596640408039093, -0.06769045442342758, 0.08294980227947235, -0.06872808933258057, 0.016174906864762306, 0.007702154573053122, 0.060121823102235794, 0.06244124099612236, 0.04361120983958244, -0.019715074449777603, -0.01293229591101408, -0.060701675713062286, 0.02526947669684887, 0.03637827932834625, 0.038056809455156326, -0.0018368437886238098, 0.006443258840590715, -0.04861627519130707, -0.11871770769357681, -0.02786356583237648, 0.04031519219279289, 0.0465715229511261, -0.039430148899555206, -0.0266580767929554, 0.011635251343250275, -0.00928531214594841, 0.0025425883941352367, 0.009979612194001675, -0.001072922721505165, -0.07635776698589325, -0.0024796435609459877, -0.0038072066381573677, 0.050142209976911545, 0.0036088351625949144, 0.04528973624110222, -0.030167726799845695, 0.040040522813797, -0.0073664491064846516, -0.0659814104437828, -0.018128102645277977, 0.0599081926047802, -0.029725205153226852, 0.008667307905852795, -0.02916060946881771, 0.018234917894005775, -0.041078

In [5]:
len(embedings)

384

#  Wrangle dataset

In [6]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [7]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [8]:
texts = df['text'].tolist()

In [9]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [10]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

In [11]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

# Generate Embeddings

In [12]:
output = client.embeddings.create(input = texts,model=model_name)

In [13]:
embeddings = output.data

In [14]:
embeddings

[Embedding(embedding=[-0.014101148582994938, -0.06446239352226257, 0.003779000835493207, 0.0047423504292964935, 0.03017096407711506, -0.04563033953309059, 0.03717575594782829, 0.010736101306974888, -0.027744466438889503, -0.05091064050793648, -0.05237569287419319, -0.0336657278239727, -0.00046569842379540205, -0.0437684990465641, 0.03751149773597717, -0.01719912700355053, -0.019473014399409294, -0.08063903450965881, -0.0758776068687439, -0.007973863743245602, 0.04840783774852753, -0.061227064579725266, -0.10743732005357742, -0.0370231457054615, 0.03555808961391449, -0.04856044799089432, 0.08778117597103119, -0.019824016839265823, 0.0017864888068288565, -0.13075610995292664, -0.012941313907504082, 0.009805181995034218, -0.02798864245414734, -0.025836843997240067, -0.03503921627998352, 0.0010482388315722346, 0.06751459091901779, -0.0759386494755745, 0.04703434929251671, 0.000599947408773005, -0.0025333231315016747, 0.02980469912290573, -0.06415717303752899, -0.002111738547682762, -0.0068

# Push data to vector database

In [15]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [16]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

# Get Closest documents

In [17]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [18]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.734824479,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 